In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import time
import numpy as np

import sys
sys.path.append('../mlrose')
from mlrose_hiive.generators.queens_generator import QueensGenerator
from mlrose_hiive import RHCRunner

In [ ]:
ALGORITHM = 'rhc'
PROBLEM_SIZE = [10, 50, 100, 200, 500]
MAX_ITERATIONS = 1000
MAX_ATTEMPTS = 100
NUM_RUNS = 3

# RESTARTS = [1, 5, 10, 20]
RESTARTS = [50]

In [ ]:
def get_max_size(problem_size):
    if problem_size <= 1:
        return 0
    if problem_size == 2:
        return 1
    return 3*(problem_size-2)
OPT_SCORE = get_max_size(PROBLEM_SIZE)
OPT_SCORE

In [ ]:
# run_i = 6
run_i = int(time.time())
problem = QueensGenerator().generate(seed=run_i, size=PROBLEM_SIZE, maximize=True)

runner = RHCRunner(
    problem=problem,
    experiment_name='RHC',
    output_directory='.',
    seed=run_i,
    iteration_list=[MAX_ITERATIONS],
    max_attempts=MAX_ATTEMPTS,
    restart_list=[1],
)
x, df_run_curves = runner.run()

In [ ]:
print(df_run_curves['Fitness'].max())
df_run_curves.head()

In [ ]:
df_path = f'metrics/{ALGORITHM}.csv'
os.makedirs(os.path.dirname(df_path), exist_ok=True)
if not os.path.exists(df_path):
    all_df = pd.DataFrame(columns=[
        # runner columns
        'Iteration', 'Time', 'Fitness', 'FEvals', 'Restarts', 'max_iters', 'current_restart',
        # problem config
        'problem_size', 'max_attempts',
        # run config
        'run_number', 'group_timestamp', 'total_runtime',
    ])
    all_df.to_csv(df_path, index=False)
all_df = pd.read_csv(df_path)

In [ ]:
for restarts in RESTARTS:
    print(f"restarts: {restarts}")

    if (
        (all_df['problem_size'] == PROBLEM_SIZE) &
        (all_df['Restarts'] == restarts) &
        (all_df['max_attempts'] == MAX_ATTEMPTS)
    ).any():
        print('Already run')
        continue

    group_timestamp = int(time.time())
    time.sleep(1)
    max_fitnesses = []
    for run_i in tqdm(range(NUM_RUNS)):
        t_start = time.perf_counter()
        problem = QueensGenerator().generate(seed=run_i, size=PROBLEM_SIZE, maximize=True)
        runner = RHCRunner(
            problem=problem,
            experiment_name='RHC',
            output_directory='.',
            seed=run_i,
            iteration_list=[MAX_ITERATIONS],
            max_attempts=MAX_ATTEMPTS,
            restart_list=[restarts],
        )
        _, df_run_curves = runner.run()

        df_run_curves['problem_size'] = PROBLEM_SIZE
        df_run_curves['max_attempts'] = MAX_ATTEMPTS

        df_run_curves['Restarts'] = restarts
        df_run_curves['max_iters'] = MAX_ITERATIONS

        df_run_curves['run_number'] = run_i
        df_run_curves['group_timestamp'] = group_timestamp
        df_run_curves['num_edges'] = problem.length

        max_fitnesses.append(df_run_curves['Fitness'].max())

        t_end = time.perf_counter()
        df_run_curves['total_runtime'] = t_end - t_start

        all_df = pd.concat([all_df, df_run_curves])

    print(f"Max Fitnesses: {np.mean(max_fitnesses)}")

    all_df['optimal_fitness'] = OPT_SCORE
    all_df.reset_index(inplace=True, drop=True)
    all_df.to_csv(df_path, index=False)

In [ ]:
all_df.columns

In [ ]:
agg_df = pd.DataFrame()
for group_timestamp in all_df['group_timestamp'].unique():
    df = all_df[all_df['group_timestamp'] == group_timestamp]
    max_fit = df['Fitness'].max()
    total_iters = df['Iteration'].max()
    max_fevals = df['FEvals'].max()
    # df.drop(columns=['Fitness', 'Iteration', 'FEvals', 'Time'], inplace=True)
    df.drop(columns=['Fitness', 'Iteration', 'FEvals', 'Time', 'current_restart'], inplace=True)
    assert(len(df) > NUM_RUNS)
    df.drop_duplicates(inplace=True)
    assert(len(df) == NUM_RUNS)
    df['max_fitness'] = max_fit
    df['total_iterations'] = total_iters
    df['max_fevals'] = max_fevals
    agg_df = pd.concat([agg_df, df])
agg_df.reset_index(inplace=True, drop=True)

In [ ]:
df_path = f'metrics/{ALGORITHM}_agg.csv'
agg_df.to_csv(df_path, index=False)